# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [38]:
! pip install -U langchain

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [39]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [40]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

cs_template = """You are an experienced computer scientist.
You excel at solving problems in the field of computer science, whether theoretical or practical applications.
When faced with a problem outside your knowledge, you seek help from other experts or recommend reliable resources.

Here's a question:
{input}"""

bio_template = """You are a knowledgeable biologist.
You are skilled at answering questions related to biology, from cellular processes to ecosystems.
When faced with a question outside your expertise, you seek help from other experts or recommend reliable resources.

Here's a question:
{input}"""
lang_literature_template = """You are a well-versed scholar in Chinese language and literature.
You excel at discussing and analyzing the nuances of Chinese language, poetry, prose, and literary history.
When encountering a challenging topic, you consult with other scholars or suggest authoritative texts for further reading.

Here's a topic:
{input}"""


In [41]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "Computer Science",
        "description": "To answer questions and solve problems that related with computer science",
        "prompt_template": cs_template,
    },
    {
        "name": "Biology",
        "description": "To answer questions and solve problems related to biology",
        "prompt_template": bio_template,
    },
    {
        "name": "Chinese Language and Literature",
        "description": "To discuss and analyze topics related to Chinese language and literature",
        "prompt_template": lang_literature_template,
    }
]

In [42]:
import os
# Set the API key as an environment variable
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [43]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [44]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [45]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [46]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [47]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', 'Computer Science: To answer questions and solve problems that related with computer science', 'Biology: To answer questions and solve problems related to biology', 'Chinese Language and Literature: To discuss and analyze topics related to Chinese language and literature']


In [48]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
Computer Science: To answer questions and solve problems that related with computer science
Biology: To answer questions and solve problems related to biology
Chinese Language and Literature: To discuss and analyze topics related to Chinese language and literature


In [49]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [50]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
Computer Science: To answer questions and solve problems that related wit

In [51]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [52]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': 'What is blackbody radiation?'}
> Finished chain.
{'input': 'What is blackbody radiation?', 'text': '\n\n黑体辐射是指一个完美吸收一切辐射的物体所发出的热辐射。根据普朗克定律，黑体辐射的能量分布与温度有关，即温度越高，发出的辐射更强，波长也更短。这种现象被广泛应用于热力学和宇宙学领域。'}


In [53]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除?'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除?', 'text': '\n\n大于40的第一个质数是41。41加一为42，42除以3余数为0，所以满足条件。'}


In [54]:
router_chain.verbose = True

In [55]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞'}
> Finished chain.
{'input': '黑洞', 'text': '是什么？\n\n黑洞是宇宙中一种极度强大的引力场，它的存在是基于爱因斯坦的广义相对论理论。当一个恒星耗尽了它的燃料，它会坍缩成一个无比小而密度极高的物体，这就是黑洞。它的引力场非常强大，甚至连光也无法逃脱，所以它被称为“黑洞”。它能够吞噬一切物质和能量，包括光线，因此它也被称为“自然界的吞噬者”。黑洞在宇宙中起着重要的作用，它们可以影响周围的星系和宇宙结构。目前，科学家们正在努力研究黑洞的性质和它们对宇宙'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [56]:
print(chain.invoke("what is the basic composition of a human?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Biology: {'input': 'what is the basic composition of a human?'}
> Finished chain.
{'input': 'what is the basic composition of a human?', 'text': '\n\nA human is composed of various elements, including oxygen (65%), carbon (18.5%), hydrogen (9.5%), nitrogen (3.2%), calcium (1.5%), phosphorus (1%), and other trace elements such as potassium, sulfur, sodium, chlorine, and magnesium. These elements make up our cells, tissues, organs, and ultimately our entire body. Additionally, water makes up about 60% of our body weight. Our body also contains proteins, lipids, carbohydrates, nucleic acids, and other biomolecules that are essential for our survival. '}


In [57]:
print(chain.invoke("孕育一只老鼠的幼崽周期通常为多久？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Biology: {'input': 'How long is the gestation period of a mouse pup?'}
> Finished chain.
{'input': 'How long is the gestation period of a mouse pup?', 'text': '\n\nThe gestation period of a mouse pup is approximately 19-21 days. However, this can vary slightly depending on the species and individual mouse.'}


In [58]:
print(chain.invoke("什么是计算机网络？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Computer Science: {'input': '什么是计算机网络?'}
> Finished chain.
{'input': '什么是计算机网络?', 'text': '\n\n计算机网络是指多台计算机通过通信线路或其他通信设备相互连接起来，以实现数据交换和资源共享的系统。它可以将地理位置不同的计算机连接起来，使它们可以相互通信，并共享数据和资源，从而构成一个巨大的网络。计算机网络的主要目的是实现信息的传输和共享，提高信息传输的速度和效率，使用户可以方便地获取和利用信息。它是现代信息社会的基础设施，广泛应用于互联网、局域网、城域网和广域网等各种网络环境中。'}


In [61]:
print(chain.invoke("简单介绍一下中文的发展历史"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Chinese Language and Literature: {'input': '简单介绍一下中文的发展历史'}
> Finished chain.
{'input': '简单介绍一下中文的发展历史', 'text': '\n\n中文是一种古老而复杂的语言，在中国有着悠久的发展历史。根据考古发现，早在约6000年前的仰韶文化时期，中国就已经出现了最早的文字，称为“甲骨文”。随着时代的发展，中国的文字也不断演变，经历了“金文”、“小篆”、“隶书”等不同的阶段。\n\n到了秦朝时期，秦始皇统一中国后，实行了统一的文字，即今天我们所使用的“汉字”。汉字的发展也经历了不同的阶段，如隋唐时期的“楷书”和宋元时期的“行书”。\n\n随着社会的变革，中国的文字也在不断发展。在清朝时期，出现了“楷体”，并逐渐成为主流的书写'}
